In [0]:
df = spark.read\
    .format("csv")\
        .option("header", "true")\
            .option("inferschema","true")\
                .load("/FileStore/invoices.csv")

In [0]:
df.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|     null|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 8.26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 8.

In [0]:
df.selectExpr(
    "count(*) as Total_Count",
    "sum(Quantity) as Total_Quantity",
    "approx_count_distinct(InvoiceNo) as Distinct_Invoice",
    "avg(unitPrice) as Avg_Price"
).show()

+-----------+--------------+----------------+-----------------+
|Total_Count|Total_Quantity|Distinct_Invoice|        Avg_Price|
+-----------+--------------+----------------+-----------------+
|     541909|       5176450|           26470|4.611113626089712|
+-----------+--------------+----------------+-----------------+



In [0]:
from pyspark.sql import functions as f

In [0]:
df.select(
          f.count("*").alias("Total_count"),
          f.sum("Quantity").alias("Total_Quantity"),
          f.countDistinct("InvoiceNo").alias("InvoiceDistinct"),
          f.avg("unitPrice").alias("Avg_price") 
          ).show()

+-----------+--------------+---------------+----------------+
|Total_count|Total_Quantity|InvoiceDistinct|       Avg_price|
+-----------+--------------+---------------+----------------+
|     541909|       5176450|          25900|4.61111362608848|
+-----------+--------------+---------------+----------------+



In [0]:
df.createOrReplaceempView("table")

In [0]:
df2 = spark.sql(
    """
    SELECT Country , InvoiceNo,
    sum(Quantity) as Total_Quantity,
    sum(Quantity*UnitPrice) as InvoiceValue
    from table
    Group By Country, InvoiceNo
    """
)

In [0]:
df2.show()

+--------------+---------+--------------+-------------------+
|       Country|InvoiceNo|Total_Quantity|       InvoiceValue|
+--------------+---------+--------------+-------------------+
|United Kingdom|   536446|           329|             440.89|
|United Kingdom|   536508|           216|             155.52|
|United Kingdom|   537018|            -3|                0.0|
|United Kingdom|   537401|           -24|                0.0|
|United Kingdom|   537811|            74|             268.86|
|United Kingdom|  C537824|            -2|-14.899999999999999|
|United Kingdom|   538895|           370|             247.38|
|United Kingdom|   540453|           341| 302.44999999999993|
|United Kingdom|   541291|           217| 305.81000000000006|
|United Kingdom|   536627|            64| 306.20000000000005|
|United Kingdom|   537224|           700| 1415.9700000000003|
|United Kingdom|   537230|             1|               2.95|
|United Kingdom|   537682|            72|               99.6|
|United 

In [0]:
df2 = df.groupBy("Country", "InvoiceNo")\
    .agg(f.count("*").alias("Total_count"),
          f.sum("Quantity").alias("Total_Quantity"),
          f.countDistinct("InvoiceNo").alias("InvoiceDistinct"),
           f.expr("round(sum(Quantity*UnitPrice),2) as InvoiceValue"),
          f.avg("unitPrice").alias("Avg_price") )
    
df2.show()

+--------------+---------+-----------+--------------+---------------+------------+------------------+
|       Country|InvoiceNo|Total_count|Total_Quantity|InvoiceDistinct|InvoiceValue|         Avg_price|
+--------------+---------+-----------+--------------+---------------+------------+------------------+
|United Kingdom|   536553|          1|             3|              1|         0.0|               0.0|
|United Kingdom|   536574|          1|             3|              1|       17.85|              5.95|
|United Kingdom|   537397|         10|           164|              1|       244.2|1.7700000000000002|
|United Kingdom|   537636|          3|            32|              1|       172.0| 7.816666666666666|
|United Kingdom|   537655|          8|            57|              1|      111.15|3.0749999999999997|
|United Kingdom|   537885|         16|            69|              1|      258.75|          5.471875|
|United Kingdom|   538651|         23|            78|              1|      280.94|

In [0]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
new_df1 = df.withColumn("Date", f.to_date(f.col("InvoiceDate"), "dd-MM-yyyy H.mm"))


In [0]:
new_df1.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Date: date (nullable = true)



In [0]:
new_df1= df.withColumn("week of Number",f.weekofyear(f.col("InvoiceDate")))

In [0]:
new_df1.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|week of Number|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+--------------+
|   536365|     null|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|          null|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|          null|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|          null|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|          null|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|          null|
|   536365|    22752|SET 7 BABUSHKA NE...|      